# Multi-Node Rendering with Dask-MPI and Dask.Array
If you looked at the Array.ipynb example, you saw server-side rendering driven by Jupyter's python kernel, but we can also drive the renderer with Dask. What's more, we can use a cluster of Dask-MPI workers to distribute the rendering across multiple GPUs, or even multiple nodes. It takes a little more setup than the single-node, Jupyter driven case but, with large enough data, sometimes you just want to be able to to throw hardware at the problem.

Before we get started, we'll need a running Dask-MPI cluster. The scheduler process and worker processes are launched via separate `mpiexec`/`mpirun` calls--the Dask scheduler doesn't participate as a worker, which means that one of our ranks would be missing from `COMM_WORLD` if we launched the scheduler and workers together. The root project directory contains a script which demonstrates the syntax for launching a dask-mpi cluster.

Once the cluster is ready, our next step is to connect to the Dask scheduler from the Jupyter client. Here we're using a description file (`scheduler.json`) output by the scheduler process at startup, but you could just as easily connect to it via URL.

In [ ]:
from dask.distributed import Client
client = Client(scheduler_file='/tmp/scheduler.json', set_as_default=True) #Connect to the Dask scheduler

N = len(client.scheduler_info()['workers']) #Get the number of workers in our cluster
print("Connected to cluster with", N, "workers")

We're using the same data here as in the single-node array example, but this time we use Dask to load multiple chunks of it in parallel. We launch 128 tasks, each of which will load a single chunk in a lazy fashion. Once the data is loaded, we rechunk and rebalance in order to create a 1:1 mapping of chunks to workers.

In [ ]:
# Make sure we're able to import urllib.request
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import os

# Check for a cached copy of the dataset, and download it if needed
# NOTE: we use an absolute path to the data because the cluster may not be running in the same working directory as the Jupyter kernel
filename = '/tmp/Supernova_1350.dat'
if not os.path.exists(filename):
    url = 'https://data.kitware.com/api/v1/item/5bdc652f8d777f21798533f0/download'
    urlretrieve(url, filename)


import numpy as np

### Load the array in parallel
def getChunk(fn, n, shape, dtype):
    sz = shape[0]*shape[1]*shape[2]
    with open(fn, 'rb') as f:
        f.seek(n*sz*4)
        return np.fromfile(f, dtype=dtype, count=sz).reshape(shape)

import dask
import dask.array as da
from dask.distributed import wait
dims,dtype = [432,432,432],np.float32

shape = [1,dims[1],dims[2]]
parts = [da.from_delayed(dask.delayed(getChunk)(filename,n,shape,dtype),shape,dtype) for n in range(0,dims[0])]

ar = da.concatenate(parts) #combine array parts into a single dask array
ar = ar.rechunk([dims[0]/N, dims[1], dims[2]]).persist() #rechunk to get 1 block per worker
wait(ar) #wait on the load+concat+rechunk to make the data resident on workers
client.rebalance() #redistribute data evenly

Now that we're set up, we're ready to launch a `PVRenderActor` per worker. We use Dask Actors for rendering because they can maintain their own internal state, and can wait around as background threads on our workers until we request a new frame. This means we initialize our state just once at the start of rendering, and still have our cluster free for further data processing.

Actually launching the Actors is a two step process. First, `client.map(actor=True)` instantiates our Actor objects across all of the workers. Note the `range(N)` (`N` is the number of workers in our cluster) in the call, which makes sure that Dask spawns the one task/worker that we want. `client.map` just returns futures, so we need to gather the results back to the Jupyter kernel in order to work with them. The final output is a list of `dask.distributed.actor.Actor` that can be used to access the Actors across all of the ranks.

In [ ]:
from ipyparaview import PVRenderActor
renderers = client.gather(client.map(PVRenderActor, range(N), actor=True))

Similar to the single-node rendering example, we need to set up some ParaView state. The difference here is that the state has to be configured on every worker, instead of locally in the Jupyter kernel. The state script is wrapped in a function object, which is then passed to and run on every worker.

In [ ]:
# Define a function for remote execution that will set up the ParaView state
def workerState(self, arr):
        import numpy as np
        import vtk
        from vtk.util import numpy_support as vtknp
        
        #Use the undocumented block slicing to get the block for this rank
        wdims = arr.shape[::-1]
        ar = arr.blocks[self.rank].compute()
        dims = ar.shape[::-1]
        print("Rank", self.rank, "has array with local/global dims", dims, wdims)
        ar = np.reshape(ar, dims[0]*dims[1]*dims[2])
        
        ext = [0,dims[0]-1, 0,dims[1]-1, max(self.rank*dims[2]-1,0),(self.rank+1)*dims[2]-1]
        wext = [0,wdims[0]-1, 0,wdims[1]-1, 0,wdims[2]-1]

        vtkimg = vtk.vtkImageData()
        vtkimg.Initialize()
        vtkimg.SetExtent(ext)
        vtkimg.SetSpacing([1,1,1])
        
        #set the extent for the whole dataset
        vi = vtk.vtkInformation()
        vtkimg.CopyInformationToPipeline(vi)
        vi.Set(vtk.vtkStreamingDemandDrivenPipeline.WHOLE_EXTENT(), wext[0],wext[1],wext[2],wext[3],wext[4],wext[5])
        vtkimg.CopyInformationFromPipeline(vi)

        varnm = 'E' #'E' is entropy for this data
        vtkarr = vtknp.numpy_to_vtk(ar)
        vtkarr.SetName(varnm)
        vtkimg.GetPointData().AddArray(vtkarr)
        vtkimg.GetPointData().SetScalars(vtkarr)

        self.TP = self.pvs.TrivialProducer()
        self.TP.GetClientSideObject().SetOutput(vtkimg)
        self.TP.UpdatePipeline()
        
        #initializae some renderer settings
        self.renv.ViewSize = [800, 500]
        self.renv.CameraPosition = [650,0,0]
        self.renv.Background = [0.0, 0.0, 0.0]

        #create a display object for the data, and set it to volume render
        self.TPDisplay = self.pvs.Show(self.TP, self.renv)
        ePWF,eLUT = self.pvs.GetOpacityTransferFunction(varnm), self.pvs.GetColorTransferFunction(varnm)
        eLUT.RGBPoints = [3.0241666020214752e-15, 0.0392156862745098, 1.0, 0.9686274509803922, 0.05988497659564321, 0.0392156862745098, 1.0, 0.9686274509803922, 0.06215288117527962, 0.0, 0.0, 0.0, 0.06337877362966537, 0.0, 0.0, 0.0, 0.06871142238378525, 0.901960784314, 0.0, 0.0, 0.0716535672545433, 0.901960784314, 0.901960784314, 0.0, 0.08403510600328445, 0.9882352941176471, 0.9882352941176471, 0.9882352941176471, 0.11376306414604187, 1.0, 1.0, 1.0]
        eLUT.ColorSpace = 'RGB'
        ePWF.Points = [3.0241666020214752e-15, 0.0, 0.5, 0.0, 0.032547514885663986, 0.0, 0.5, 0.0, 0.03309916704893112, 0.3529411852359772, 0.5, 0.0, 0.03346693515777588, 0.0, 0.5, 0.0, 0.06215288117527962, 0.0, 0.5, 0.0, 0.06779199838638306, 0.05882352963089943, 0.8863638639450073, 0.0, 0.07698621600866318, 0.11029411852359772, 0.5, 0.0, 0.08078648895025253, 0.04411764815449715, 0.5, 0.0, 0.08244144916534424, 0.4852941334247589, 0.5, 0.0, 0.08378992974758148, 0.0, 0.5, 0.0, 0.08746761322713148, 0.0, 0.5, 0.0, 0.09617146849632263, 0.0, 0.5, 0.0, 0.10965631902217865, 0.4117647111415863, 0.5, 0.0, 0.11376306414604187, 1.0, 0.5, 0.0]

        # trace defaults for the display properties.
        self.TPDisplay.Representation = 'Volume'
        self.TPDisplay.ColorArrayName = ['POINTS', varnm]
        self.TPDisplay.LookupTable = self.pvs.GetColorTransferFunction(varnm)
        self.TPDisplay.OpacityArray = ['POINTS', varnm]
        self.TPDisplay.ScalarOpacityFunction = self.pvs.GetOpacityTransferFunction(varnm)


# Submit the setup function for execution on Dask workers
wait([r.run(workerState, [ar]) for r in renderers])

And that's it! We can now pop up a PVDisplay widget and do some interactive rendering. Note that we pass in the list of render Actors, instead of a `RenderView` object as we did in the single-node example.

In [ ]:
from ipyparaview.widgets import PVDisplay
w = PVDisplay(renderers)
display(w)